In [36]:
# Khai báo sử dụng các thư viện. Thực hiện chạy ngay trên CPU, do đó, chỉ thực hiện mô phỏng, nếu ổn định sẽ thực hiện chạy trên kaggle
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sys

import regex as re
import os
import hashlib
from datetime import datetime
import hashlib
import copy

from tabulate import tabulate # Thư viện sử dụng hiển thị dữ liệu dạng bảng cho DataFrame

from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [37]:
# Chỉnh sửa class Logcluster:
from Root_Drain import LogParser, Node, Logcluster            # Source code cũ của Drain

class Logcluster3(Logcluster):
    """
    Class được chỉnh sửa lại, bao gồm một số thông số như sau:
    
    Thuộc tính:
    ----------
        - `logTemplate`: Template đặc trưng đại diện cho nhóm log đó, ["The1", "the2", ...]
        - `logIDL`     : Danh sách các ID message log thuộc nhóm log trên, [1,2,3,4,5, ...] 
        - `levelL`     : Danh sách các Level (INFO, WARN, FATAL, ERROR) mà nhóm log có thể biểu diễn, ["WARN", "INFO", ...]
        - `idTemplate` : Mã định danh của template đó, có thể bị thay đổi do template.
        - `idLevelTemp`: Xác định mã định danh theo dạng {"INFO": "L1", ...} 
    """
    def __init__(self, logTemplate="", levelL=None, logIDL=None):
        self.logTemplate = logTemplate
        if logIDL is None:
            logIDL = []
        self.logIDL = logIDL
        self.levelL = list(levelL)
        self.idTemplate = None
        self.idLevelTemp = {"TEMP":None}
        
    def addIDLevel(self, logLevel):
        self.levelL.append(logLevel)



In [38]:
# Thực hiện chỉnh sửa lại 2 phương thức outputResult() và parser() của Drain:
# Kế thừa Drain bằng cách tạo ra class Drain2:
class LogParser2(LogParser):
    """
    Kế thừa từ LogParser được sử dụng để thực hiện sửa đổi lại sao cho phù hợp.
    """
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)       # Đọc thêm: https://viblo.asia/p/python-args-va-kwargs-gDVK2pdnlLj
        self.rootNode = Node()
        self.list_logClust = []
        self.list_eventsID = {}
        self.df_TemplateL = None
        self.list_counts = {"TEMP":0}
    
    def convert_unixtime(self, row):
        """
        Phương thức trích xuất thời gian theo dạng chuẩn UNIX
        """
        date_str = row["Date"]
        time_str = row["Time"]
        datetime_str = date_str[:2] + "-" + date_str[2:4] + "-" + date_str[4:] + " " + time_str[:2] + ":" + time_str[2:4] + ":" + time_str[4:]
        unixtime = datetime.strptime(datetime_str, "%m-%y-%d %H:%M:%S").timestamp()
        return int(unixtime)
    
    # def extract_block_id(self, parameter_list):
    #     """
    #     Phương thức trích xuất blockID cho từng thông điệp log
    #     """
    #     block_ids = []
    #     pattern = re.compile(r'\bblk_-?\d+\b')  # Biểu thức chính quy để tìm blockID
    #     for param in parameter_list:
    #         matches = pattern.findall(param)
    #         if matches:  # Kiểm tra nếu tìm thấy blockID
    #             block_ids.extend(matches)
    #     return block_ids

    def extract_block_id(self, parameter_list):
        """
        Phương thức trích xuất blockID cho từng thông điệp log

        Trả về:
            - Chuỗi blockID đầu tiên được tìm thấy hoặc `NaN` nếu không tìm thấy.
        """
        pattern = re.compile(r'\bblk_-?\d+\b')  # Biểu thức chính quy để tìm blockID
        for param in parameter_list:
            matches = pattern.findall(param)
            if matches:  # Kiểm tra nếu tìm thấy blockID
                return matches[0]
        return np.nan

    def extractByBlockID(self):
        """
            Trích xuất ra quy trình template sử dụng theo BlockID
        """
        unique_values = self.df_log['BlockID'].unique()
        unique_values = [x for x in unique_values if pd.notna(x)]

        res = {}

        for value in unique_values:
            filtered_df = self.df_log[self.df_log['BlockID'] == value].sort_values(by='UnixTime')
            res[value] = filtered_df['IDLevel'].values.tolist()
        
        return res
    
    def outputResult(self, logClustL):
        """
        Thực hiện in thông tin kết quả ra ngoài sau khi hoàn thành quá trình phân tích
        
        Tham số:
        --------
            - `logClustL`   : Danh sách các nhóm log đã thu thập được trong quá trình phân tách
        """
        # ----------- Khai báo các biến sử dụng ----------#
        log_templates = [0] * self.df_log.shape[0]
        log_templateids = [0] * self.df_log.shape[0]
        list_events = []                            # Mảng list_events lưu giữ các giá trị template
        dictionary_events = self.list_eventsID      # Dictionary lưu giữ các đối tượng idTemplate và các thông số của nó: "idTemplate": {"TEMP":"L5", "INFO":"I2", ...}
    
        # --------------Thực hiện phân tích--------------#
        # * Duyệt qua từng đối tượng Logcluster để thêm các giá trị vào từng biến trên
        for logClust in logClustL:
            template_str = " ".join(logClust.logTemplate)
            occurrence = len(logClust.logIDL)
            template_id = hashlib.md5(template_str.encode("utf-8")).hexdigest()[0:8]
            logClust.idTemplate = template_id   # không cần cũng được
            for logID in logClust.logIDL:
                logID -= 1
                log_templates[logID] = template_str
                log_templateids[logID] = template_id
            logClust.logIDL=[]
            list_events.append([template_id, template_str, occurrence, logClust.levelL, logClust.idLevelTemp])
            if template_id not in dictionary_events:
                dictionary_events[template_id] = logClust.idLevelTemp

        # DataFrame df_event lưu giữ các thông tin của Template
        df_event = pd.DataFrame(
            list_events, columns=["EventId", "EventTemplate", "Occurrences", "logLevel", "IDlogLevel"]
        )
        # varL1.df_all_events = df_event
        
        # DataFrame df_log lưu giữ các thông tin của từng dòng thông điệp log
        self.df_log["EventId"] = log_templateids
        self.df_log["EventTemplate"] = log_templates
        
        # Chuyển đổi thời gian theo dạng UnixTime:
        # Tạo thêm cột "UnixTime" trong DataFrame: df_log
        self.df_log["UnixTime"] = self.df_log.apply(self.convert_unixtime, axis=1)
                
        if self.keep_para:
            self.df_log["ParameterList"] = self.df_log.apply(
                self.get_parameter_list, 
                axis=1
            )
        self.df_log["IDLevel"] = self.df_log.apply(
            lambda row: dictionary_events.get(row["EventId"], {}).get(row["Level"], None),
            axis=1
        )
        self.df_log["LineId2"] = self.df_log.apply(
            lambda row: dictionary_events.get(row["EventId"], {}).get("TEMP", None),
            axis=1
        )
        self.df_log["BlockID"] = self.df_log["ParameterList"].apply(self.extract_block_id)
        
        result_list = self.extractByBlockID()
        result_df = pd.DataFrame(list(result_list.items()), columns=['BlockID', 'Process'])
        result_df['Process'] = result_df['Process'].apply(lambda x: ', '.join(map(str, x)))
        
        # self.df_log.to_csv(
        #     os.path.join(self.savePath, self.logName + "_structured.csv"), index=False
        # )
        result_df.to_csv(
            os.path.join(self.savePath, self.logName.replace('.log', '') + "_process.csv"),
            index=False
        )
    
    def parse(self, logName):
        """
        Phương thức thực hiện quá trình phân tích log
        
        Tham số: 
        --------
            - `logName`: Tên file đầu vào thực hiện phân tích
        """
        print("File đầu vào: " + os.path.join(self.path, logName))
        start_time = datetime.now()
        self.logName = logName

        # ------Gán lại các đối tượng sử dụng--------#
        rootNode = self.rootNode       # Địa chỉ Node gốc
        logCluL = self.list_logClust   # Lưu giữ các đối tượng nhóm log Logcluster2 trước đó.
        templateLogCluL = []           # Lưu giữ các template đã có trước đó.

        # ----------------Load dữ liệu---------------#
        self.load_data() 

        #-----------------Phân tích------------------#
        # * Gán giá trị cho danh sách đếm template:
        unique_Level = self.df_log["Level"].unique()
        for value in unique_Level:
            if value not in self.list_counts:
                self.list_counts[value] = 0

        # * Thực hiện phân tích và lấy Template cho từng dòng thông điệp log
        count = 0
        for idx, line in self.df_log.iterrows(): 
            logLevel = str(line["Level"])
            logID = line["LineId"]
            logmessageL = self.preprocess(line["Content"]).strip().split()  
            matchCluster = self.treeSearch(rootNode, logmessageL)           
            if matchCluster is None:
                newCluster = Logcluster3(logTemplate=logmessageL, levelL=[logLevel], logIDL=[logID])
                
                # - Nếu có template mới, tạo ra, tăng biến đếm, gán id level template cho mẫu:
                self.list_counts["TEMP"] += 1
                self.list_counts[logLevel] += 1
                newCluster.idLevelTemp["TEMP"] = "L"+ str(self.list_counts["TEMP"])
                newCluster.idLevelTemp[logLevel] = logLevel[0] + str(self.list_counts[logLevel])
                
                # - Thêm chúng vào cây phân tích
                logCluL.append(newCluster)
                self.addSeqToPrefixTree(rootNode, newCluster)
            else:
                # - Nếu template đã có:
                newTemplate = self.getTemplate(logmessageL, matchCluster.logTemplate)
                matchCluster.logIDL.append(logID)
                # + Nếu có sự thay đổi trong template thành một template mới, cập nhật
                if " ".join(newTemplate) != " ".join(matchCluster.logTemplate):
                    matchCluster.logTemplate = newTemplate
                    
                # + Nếu có thêm một level mới biểu diễn template này, cập nhật nó
                if logLevel not in matchCluster.levelL:
                    matchCluster.addIDLevel(logLevel)
                    self.list_counts[logLevel] += 1
                    matchCluster.idLevelTemp[logLevel] = logLevel[0] + str(self.list_counts[logLevel])
                    
            count += 1
            if count % 100000 == 0 or count == len(self.df_log):
                print("Processed {0:.1f}% of log lines.".format(count * 100.0 / len(self.df_log))) # Chỉ sửa mỗi vị trí này.

        if not os.path.exists(self.savePath):
            os.makedirs(self.savePath)
        self.outputResult(self.list_logClust)
        print("Phân tích hoàn thành! [Thời gian thực hiện: {!s}]".format(datetime.now() - start_time))


In [39]:
input_dir  = '../../Dataset/HDFS/v1/'   # Đường dẫn vào thư mục chứa file đầu vào
output_dir = 'Result/'                  # Thư mục kết quả
log_file   = 'test2.log'                 # Tên file đầu vào
log_format = '<Date> <Time> <Pid> <Level> <Component>: <Content>'  # HDFS log format
# Biểu thức chính quy
regex      = [
    r'blk_(|-)[0-9]+' , # block id
    r'(/|)([0-9]+\.){3}[0-9]+(:[0-9]+|)(:|)', # IP
    r'(?<=[^A-Za-z0-9])(\-?\+?\d+)(?=[^A-Za-z0-9])|[0-9]+$', # Numbers
]
st         = 0.5  # Ngưỡng tương đồng
depth      = 4  # Độ sâu của cây phân tích

parserObj = LogParser2(log_format, indir=input_dir, outdir=output_dir, depth=depth, st=st, rex=regex)
# parserObj.parse(log_file)

selected_columns = ['LineId', 'UnixTime', 'Level', 'EventId', 'EventTemplate', 'IDLevel', 'LineId2', 'ParameterList', 'BlockID']
for i in range(1,6):
    str_file = "test"+str(i)+".log"
    parserObj.parse(str_file)
    print(len(parserObj.list_logClust))
    print(parserObj.list_counts)
    # outputResult2(parserObj.list_logClust)
    # parserObj.df_log.loc[:, selected_columns].head(60)
    parserObj.df_log = pd.DataFrame()
    

File đầu vào: ../../Dataset/HDFS/v1/test1.log
Total lines:  200500
Processed 49.9% of log lines.
Processed 99.8% of log lines.
Processed 100.0% of log lines.
Phân tích hoàn thành! [Thời gian thực hiện: 0:03:14.305999]
25
{'TEMP': 25, 'INFO': 24, 'WARN': 1}


,LineId,UnixTime,Level,EventId,EventTemplate,IDLevel,LineId2,ParameterList,BlockID
0,1,1312896918,INFO,09a53393,Receiving block <*> src: <*> dest: <*>,I1,L1,"[blk_-1608999687919862906, /10.250.19.102:54106, /10.250.19.102:50010]",blk_-1608999687919862906
1,2,1312896918,INFO,3d91fa85,BLOCK* NameSystem.allocateBlock: <*> <*>,I2,L2,[/mnt/hadoop/mapred/system/job_200811092030_0001/job.jar. blk_-1608999687919862906],blk_-1608999687919862906
2,3,1312896919,INFO,09a53393,Receiving block <*> src: <*> dest: <*>,I1,L1,"[blk_-1608999687919862906, /10.250.10.6:40524, /10.250.10.6:50010]",blk_-1608999687919862906
3,4,1312896919,INFO,09a53393,Receiving block <*> src: <*> dest: <*>,I1,L1,"[blk_-1608999687919862906, /10.250.14.224:42420, /10.250.14.224:50010]",blk_-1608999687919862906
4,5,1312896919,INFO,d38aa58d,PacketResponder <*> for block <*> <*>,I3,L3,"[1, blk_-1608999687919862906 terminating]",blk_-1608999687919862906
5,6,1312896919,INFO,d38aa58d,PacketResponder <*> for block <*> <*>,I3,L3,"[2, blk_-1608999687919862906 terminating]",blk_-1608999687919862906
6,7,1312896919,INFO,e3df2680,Received block <*> of size <*> from <*>,I4,L4,"[blk_-1608999687919862906, 91178, /10.250.10.6]",blk_-1608999687919862906
7,8,1312896919,INFO,e3df2680,Received block <*> of size <*> from <*>,I4,L4,"[blk_-1608999687919862906, 91178, /10.250.19.102]",blk_-1608999687919862906
8,9,1312896919,INFO,d38aa58d,PacketResponder <*> for block <*> <*>,I3,L3,"[0, blk_-1608999687919862906 terminating]",blk_-1608999687919862906
9,10,1312896919,INFO,e3df2680,Received block <*> of size <*> from <*>,I4,L4,"[blk_-1608999687919862906, 91178, /10.250.14.224]",blk_-1608999687919862906


File đầu vào: ../../Dataset/HDFS/v1/test2.log
Total lines:  397000
Processed 25.2% of log lines.
Processed 50.4% of log lines.
Processed 75.6% of log lines.
Processed 100.0% of log lines.
Phân tích hoàn thành! [Thời gian thực hiện: 0:07:00.380898]
31
{'TEMP': 31, 'INFO': 29, 'WARN': 2}


,LineId,UnixTime,Level,EventId,EventTemplate,IDLevel,LineId2,ParameterList,BlockID
0,1,1312898506,INFO,09a53393,Receiving block <*> src: <*> dest: <*>,I1,L1,"[blk_2069265995045629658, /10.251.214.225:51936, /10.251.214.225:50010]",blk_2069265995045629658
1,2,1312898506,INFO,09a53393,Receiving block <*> src: <*> dest: <*>,I1,L1,"[blk_7033524934223774239, /10.251.203.166:44587, /10.251.203.166:50010]",blk_7033524934223774239
2,3,1312898506,INFO,09a53393,Receiving block <*> src: <*> dest: <*>,I1,L1,"[blk_-7896665939960869356, /10.251.199.225:48478, /10.251.199.225:50010]",blk_-7896665939960869356
3,4,1312898506,INFO,09a53393,Receiving block <*> src: <*> dest: <*>,I1,L1,"[blk_4026107206139429967, /10.251.198.196:39719, /10.251.198.196:50010]",blk_4026107206139429967
4,5,1312898506,INFO,09a53393,Receiving block <*> src: <*> dest: <*>,I1,L1,"[blk_-4310279540387136670, /10.251.74.134:48242, /10.251.74.134:50010]",blk_-4310279540387136670
5,6,1312898506,INFO,09a53393,Receiving block <*> src: <*> dest: <*>,I1,L1,"[blk_-7896665939960869356, /10.250.7.230:34812, /10.250.7.230:50010]",blk_-7896665939960869356
6,7,1312898506,INFO,32777b38,Verification succeeded for <*>,I13,L13,[blk_4766071777347780487],blk_4766071777347780487
7,8,1312898506,INFO,5d5de21c,BLOCK* NameSystem.addStoredBlock: blockMap updated: <*> is added to <*> size <*>,I5,L5,"[10.251.67.225:50010, blk_964560911117427908, 67108864]",blk_964560911117427908
8,9,1312898506,INFO,3d91fa85,BLOCK* NameSystem.allocateBlock: <*> <*>,I2,L2,[/user/root/rand/_temporary/_task_200811092030_0001_m_001014_0/part-01014. blk_-7896665939960869356],blk_-7896665939960869356
9,10,1312898506,INFO,5d5de21c,BLOCK* NameSystem.addStoredBlock: blockMap updated: <*> is added to <*> size <*>,I5,L5,"[10.250.10.213:50010, blk_334442190204489829, 67108864]",blk_334442190204489829


File đầu vào: ../../Dataset/HDFS/v1/test3.log
Total lines:  200500
Processed 49.9% of log lines.
Processed 99.8% of log lines.
Processed 100.0% of log lines.
Phân tích hoàn thành! [Thời gian thực hiện: 0:06:47.559079]
35
{'TEMP': 35, 'INFO': 31, 'WARN': 4}


,LineId,UnixTime,Level,EventId,EventTemplate,IDLevel,LineId2,ParameterList,BlockID
0,1,1312900040,INFO,d38aa58d,PacketResponder <*> for block <*> <*>,I3,L3,"[0, blk_5520231946856976534 terminating]",blk_5520231946856976534
1,2,1312900040,INFO,e3df2680,Received block <*> of size <*> from <*>,I4,L4,"[blk_5520231946856976534, 67108864, /10.251.214.130]",blk_5520231946856976534
2,3,1312900040,INFO,09a53393,Receiving block <*> src: <*> dest: <*>,I1,L1,"[blk_-173865612221864815, /10.251.39.64:35302, /10.251.39.64:50010]",blk_-173865612221864815
3,4,1312900040,INFO,09a53393,Receiving block <*> src: <*> dest: <*>,I1,L1,"[blk_8189263971486364907, /10.251.91.229:60024, /10.251.91.229:50010]",blk_8189263971486364907
4,5,1312900040,INFO,09a53393,Receiving block <*> src: <*> dest: <*>,I1,L1,"[blk_8189263971486364907, /10.250.6.214:41154, /10.250.6.214:50010]",blk_8189263971486364907
5,6,1312900040,INFO,d38aa58d,PacketResponder <*> for block <*> <*>,I3,L3,"[0, blk_-69110746221551589 terminating]",blk_-69110746221551589
6,7,1312900040,INFO,e3df2680,Received block <*> of size <*> from <*>,I4,L4,"[blk_-69110746221551589, 67108864, /10.251.195.52]",blk_-69110746221551589
7,8,1312900040,INFO,d38aa58d,PacketResponder <*> for block <*> <*>,I3,L3,"[1, blk_8128968847817392650 terminating]",blk_8128968847817392650
8,9,1312900040,INFO,e3df2680,Received block <*> of size <*> from <*>,I4,L4,"[blk_8128968847817392650, 67108864, /10.251.43.21]",blk_8128968847817392650
9,10,1312900040,INFO,09a53393,Receiving block <*> src: <*> dest: <*>,I1,L1,"[blk_7407931596270938021, /10.251.111.37:49378, /10.251.111.37:50010]",blk_7407931596270938021


File đầu vào: ../../Dataset/HDFS/v1/test4.log
Total lines:  200500
Processed 49.9% of log lines.
Processed 99.8% of log lines.
Processed 100.0% of log lines.
Phân tích hoàn thành! [Thời gian thực hiện: 0:04:29.125346]
36
{'TEMP': 36, 'INFO': 32, 'WARN': 4}


,LineId,UnixTime,Level,EventId,EventTemplate,IDLevel,LineId2,ParameterList,BlockID
0,1,1312904792,WARN,81cee340,<*>Got exception while serving <*> to <*>,W4,L35,"[10.250.7.146:50010:, blk_-4029631711454061366, /10.251.70.5:]",blk_-4029631711454061366
1,2,1312904792,WARN,81cee340,<*>Got exception while serving <*> to <*>,W4,L35,"[10.250.7.146:50010:, blk_-8476475569159206015, /10.251.67.113:]",blk_-8476475569159206015
2,3,1312904792,WARN,81cee340,<*>Got exception while serving <*> to <*>,W4,L35,"[10.251.106.50:50010:, blk_8923633946608121410, /10.251.198.33:]",blk_8923633946608121410
3,4,1312904792,WARN,81cee340,<*>Got exception while serving <*> to <*>,W4,L35,"[10.250.7.32:50010:, blk_2687443463098456318, /10.251.123.195:]",blk_2687443463098456318
4,5,1312904792,WARN,81cee340,<*>Got exception while serving <*> to <*>,W4,L35,"[10.251.110.196:50010:, blk_2066326363374916601, /10.251.110.196:]",blk_2066326363374916601
5,6,1312904792,WARN,81cee340,<*>Got exception while serving <*> to <*>,W4,L35,"[10.251.123.99:50010:, blk_-967145856473901804, /10.251.123.99:]",blk_-967145856473901804
6,7,1312904792,WARN,81cee340,<*>Got exception while serving <*> to <*>,W4,L35,"[10.251.90.239:50010:, blk_2689893945672917905, /10.251.30.85:]",blk_2689893945672917905
7,8,1312904792,WARN,81cee340,<*>Got exception while serving <*> to <*>,W4,L35,"[10.251.199.159:50010:, blk_-1151887519555844749, /10.251.111.37:]",blk_-1151887519555844749
8,9,1312904792,WARN,81cee340,<*>Got exception while serving <*> to <*>,W4,L35,"[10.251.203.129:50010:, blk_557006208986389967, /10.251.214.112:]",blk_557006208986389967
9,10,1312904792,INFO,626085d5,<*> Served block <*> to <*>,I10,L10,"[10.251.39.192:50010, blk_-2222926717389044884, /10.250.19.227]",blk_-2222926717389044884


File đầu vào: ../../Dataset/HDFS/v1/test5.log
Total lines:  198999
Processed 50.3% of log lines.
Processed 100.0% of log lines.
Phân tích hoàn thành! [Thời gian thực hiện: 0:03:29.813008]
36
{'TEMP': 36, 'INFO': 32, 'WARN': 4}


,LineId,UnixTime,Level,EventId,EventTemplate,IDLevel,LineId2,ParameterList,BlockID
0,1,1312908593,INFO,5d5de21c,BLOCK* NameSystem.addStoredBlock: blockMap updated: <*> is added to <*> size <*>,I5,L5,"[10.251.74.192:50010, blk_-3700515036488256523, 67108864]",blk_-3700515036488256523
1,2,1312908593,INFO,3d91fa85,BLOCK* NameSystem.allocateBlock: <*> <*>,I2,L2,[/user/root/sortrand/_temporary/_task_200811092030_0002_r_000302_0/part-00302. blk_966172617673485197],blk_966172617673485197
2,3,1312908593,INFO,3d91fa85,BLOCK* NameSystem.allocateBlock: <*> <*>,I2,L2,[/user/root/sortrand/_temporary/_task_200811092030_0002_r_000361_0/part-00361. blk_4802346514735118114],blk_4802346514735118114
3,4,1312908593,INFO,5d5de21c,BLOCK* NameSystem.addStoredBlock: blockMap updated: <*> is added to <*> size <*>,I5,L5,"[10.251.193.224:50010, blk_-1927602282131607595, 67108864]",blk_-1927602282131607595
4,5,1312908593,INFO,5d5de21c,BLOCK* NameSystem.addStoredBlock: blockMap updated: <*> is added to <*> size <*>,I5,L5,"[10.251.198.33:50010, blk_-6500858188442006502, 67108864]",blk_-6500858188442006502
5,6,1312908593,INFO,5d5de21c,BLOCK* NameSystem.addStoredBlock: blockMap updated: <*> is added to <*> size <*>,I5,L5,"[10.251.31.242:50010, blk_8594679221824189646, 67108864]",blk_8594679221824189646
6,7,1312908593,INFO,5d5de21c,BLOCK* NameSystem.addStoredBlock: blockMap updated: <*> is added to <*> size <*>,I5,L5,"[10.251.43.210:50010, blk_5409263674080085474, 67108864]",blk_5409263674080085474
7,8,1312908593,INFO,3d91fa85,BLOCK* NameSystem.allocateBlock: <*> <*>,I2,L2,[/user/root/sortrand/_temporary/_task_200811092030_0002_r_000082_0/part-00082. blk_-6571231412836609949],blk_-6571231412836609949
8,9,1312908593,INFO,3d91fa85,BLOCK* NameSystem.allocateBlock: <*> <*>,I2,L2,[/user/root/sortrand/_temporary/_task_200811092030_0002_r_000321_0/part-00321. blk_5489917276823252056],blk_5489917276823252056
9,10,1312908593,INFO,09a53393,Receiving block <*> src: <*> dest: <*>,I1,L1,"[blk_-4778874422986506056, /10.251.74.227:42797, /10.251.74.227:50010]",blk_-4778874422986506056
